In [8]:
from jupyter_dash import JupyterDash
import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table as dt
from dash.dependencies import Input, Output, State
import base64
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

# Importing AnimalShelter class 
from animalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "HelloKitty"
animals = AnimalShelter(username, password)

# Read initial data to display on dashboard
df = pd.DataFrame.from_records(animals.read({}))

#########################
# Dashboard Layout / View
#########################
# create dash application
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Encode image to be used in the app
image_filename = 'Grazioso Salvare Logo.png'  # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.B(html.H2('Harrielle Monestime'))),
    html.Hr(),
    html.Center(html.Img(id='customer-image', src='data:image/png;base64,{}'.format(encoded_image.decode()), alt='Grazioso Salvare Logo')),
    html.Hr(),
    
    # Filter Options
    html.Div([
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'mount'},
                {'label': 'Disaster Rescue and Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset'
        )
    ]),
    
    html.Hr(),
    
    # Data Table
    dt.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        sort_action="native",
        sort_mode="multi",
        column_selectable="single",
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    
    html.Br(),
    html.Hr(),
    
    # Charts and Maps
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Radio button filters
@app.callback(
    [Output('datatable-id', 'data'), Output('datatable-id', 'columns')],
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    # Water Rescue filter
    if filter_type == 'water':
        df = pd.DataFrame.from_records(animals.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}
        }))
    # Mountain or Wilderness Rescue filter
    elif filter_type == 'mount':
        df = pd.DataFrame.from_records(animals.read({
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}
        }))
    # Disaster Rescue or Individual Tracking filter
    elif filter_type == 'disaster':
        df = pd.DataFrame.from_records(animals.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20.0, "$lte": 300.0}
        }))
    # Reset to no filter
    else:
        df = pd.DataFrame.from_records(animals.read({}))
    
    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data = df.to_dict('records')
    
    return data, columns

# Change background color of selected columns
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    
    return [
        dcc.Graph(            
            figure = px.histogram(dff, x='breed', title='Distribution of Breeds')
        )    
    ]

# Map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, selected_rows):
    dff = pd.DataFrame.from_dict(viewData)
    
    if selected_rows is None or len(selected_rows) == 0:
        return []
    
    selected_row = selected_rows[-1]  # Get the last selected row index
    lat = dff.iloc[selected_row, 13]  # Latitude
    lon = dff.iloc[selected_row, 14]  # Longitude
    
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(dff.iloc[selected_row, 4]),  # Breed
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[selected_row, 9])  # Name of the animal
                ])
            ])
        ])
    ]

if __name__ == '__main__':
    app.run_server(debug=False)



Error reading documents: Authentication failed., full error: {'ok': 0.0, 'errmsg': 'Authentication failed.', 'code': 18, 'codeName': 'AuthenticationFailed'}


 * Running on http://127.0.0.1:8523/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Aug/2024 01:21:26] "GET /_alive_4636f277-c7b9-45b2-9cd7-3be4f7f6b64e HTTP/1.1" 200 -


Dash app running on http://127.0.0.1:8523/


127.0.0.1 - - [26/Aug/2024 01:21:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Aug/2024 01:21:31] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [26/Aug/2024 01:21:31] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [26/Aug/2024 01:21:31] "GET /_favicon.ico?v=2.8.1 HTTP/1.1" 200 -
127.0.0.1 - - [26/Aug/2024 01:21:31] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 304 -
127.0.0.1 - - [26/Aug/2024 01:21:31] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 304 -
127.0.0.1 - - [26/Aug/2024 01:21:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Aug/2024 01:21:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Aug/2024 01:21:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Aug/2024 01:21:31] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [26/Aug/2024 01:21:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Aug/2024 01:21:31] "POST /_dash-update-componen

Error reading documents: Authentication failed., full error: {'ok': 0.0, 'errmsg': 'Authentication failed.', 'code': 18, 'codeName': 'AuthenticationFailed'}


...

Error reading documents: Authentication failed., full error: {'ok': 0.0, 'errmsg': 'Authentication failed.', 'code': 18, 'codeName': 'AuthenticationFailed'}


127.0.0.1 - - [26/Aug/2024 01:21:37] "POST /_dash-update-component HTTP/1.1" 200 -


Error reading documents: Authentication failed., full error: {'ok': 0.0, 'errmsg': 'Authentication failed.', 'code': 18, 'codeName': 'AuthenticationFailed'}


127.0.0.1 - - [26/Aug/2024 01:21:38] "POST /_dash-update-component HTTP/1.1" 200 -


Error reading documents: Authentication failed., full error: {'ok': 0.0, 'errmsg': 'Authentication failed.', 'code': 18, 'codeName': 'AuthenticationFailed'}
